# This code shows how ngrok can be used with Ollama and Gradio on Google Colab Free T4.

Prepared by Partha Pratim Ray, 11 May, 2024

MIT License



In [2]:
#pciutils contains various utilities dealing with the PCI bus (primarily lspci). lspci is a hardware detection tool for system resources connected to the PCI bus

!sudo apt-get install -y pciutils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
#Install Ollama

!curl https://ollama.ai/install.sh | sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10091    0 10091    0     0  37393      0 --:--:-- --:--:-- --:--:-- 37513
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
#Install ngrok package for pythom

!pip install pyngrok

In [5]:
#Provide authentication key from ngrok account

from pyngrok import ngrok, conf
import getpass

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()


Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


In [6]:
import os
import threading
import subprocess

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])


In [13]:
#Pull llama3 model from Ollama. You can choose any Ollama supportd model here. Just change the model name

!ollama run llama3

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest 
pulling 00e1317cbf74...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 00e1317cbf74...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 00e1317cbf74...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 00e1317cbf74...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 00e1317cbf74...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 00e1317cbf74...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 00e1317cbf74...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 00e1317cbf74...   0% ▕▏    0 B/4.7 GB                  pulling ma

In [31]:
#Start the Ollama server in a separate thread:

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [19]:
# Open Ngrok tunnel to Ollama server
ollama_tunnel = ngrok.connect("11434", "http")
public_url = ollama_tunnel.public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:11434\"")


 * ngrok tunnel "https://51ba-34-168-164-169.ngrok-free.app" -> "http://127.0.0.1:11434"


In [32]:
# Run the prompt inside the prompt = """ ..."""
# It also shows the --verbose output

import requests
import json

# Function to convert nanoseconds to seconds
def ns_to_sec(ns):
    return round(ns / 1_000_000_000, 4)

# The prompt
prompt = """
Why is sky blue? Explain.
"""


url = f'{public_url}/api/chat'

# Payload for the API request
payload = {
    "model": "llama3",  # Replace with the correct model name
    "temperature": 0.7,
    "stream": False,
    "messages": [
        {"role": "system", "content": "You are an AI assistant! Provide accurate, verbose responses."},
        {"role": "user", "content": prompt}
    ]
}

# Send the request and handle the response
response = requests.post(url, json=payload)
response_content = response.content.decode('utf-8')

# Debug: Print the entire response
print("Raw Response:", response_content)

# Try to parse and handle different response scenarios
try:
    message_dict = json.loads(response_content)

    # Check if the response contains the desired content
    if 'message' in message_dict:
        # Extract specific metrics
        response_message = message_dict['message']['content']
        model_name = message_dict.get('model', 'Unknown')
        total_duration_ns = message_dict.get('total_duration', 0)
        load_duration_ns = message_dict.get('load_duration', 0)
        prompt_eval_count = message_dict.get('prompt_eval_count', 0)
        prompt_eval_duration_ns = message_dict.get('prompt_eval_duration', 0)
        eval_count = message_dict.get('eval_count', 0)
        eval_duration_ns = message_dict.get('eval_duration', 0)

        # Convert durations from nanoseconds to seconds
        total_duration = ns_to_sec(total_duration_ns) if total_duration_ns else 'Unknown'
        load_duration = ns_to_sec(load_duration_ns) if load_duration_ns else 'Unknown'
        prompt_eval_duration = ns_to_sec(prompt_eval_duration_ns) if prompt_eval_duration_ns else 'Unknown'
        eval_duration = ns_to_sec(eval_duration_ns) if eval_duration_ns else 'Unknown'

        # Calculate evaluation rates
        prompt_eval_rate = round(prompt_eval_count / prompt_eval_duration, 2) if prompt_eval_count and prompt_eval_duration != 'Unknown' else 'Unknown'
        eval_rate = round(eval_count / eval_duration, 2) if eval_count and eval_duration != 'Unknown' else 'Unknown'

        # Print the formatted response
        print("\n### Formatted Response ###")
        print(f"\nResponse:\n{response_message}")

        # Print verbose information
        print("\n### Verbose Metrics ###")
        print(f"Model Name: {model_name}")
        print(f"Total Duration: {total_duration}s")
        print(f"Load Duration: {load_duration}s")
        print(f"Prompt Eval Count: {prompt_eval_count} token(s)")
        print(f"Prompt Eval Duration: {prompt_eval_duration}s")
        print(f"Prompt Eval Rate: {prompt_eval_rate} tokens/s")
        print(f"Eval Count: {eval_count} token(s)")
        print(f"Eval Duration: {eval_duration}s")
        print(f"Eval Rate: {eval_rate} tokens/s")

    else:
        print("Response does not contain 'message'. Full response:", message_dict)

except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


Raw Response: {"model":"llama3","created_at":"2024-05-10T18:11:33.474546118Z","message":{"role":"assistant","content":"A question that has fascinated humans for centuries! The phenomenon of the sky appearing blue is a result of a complex interplay between various scientific principles and natural processes.\n\nTo begin with, light from the sun is comprised of a broad spectrum of electromagnetic radiation, commonly referred to as white light. This white light contains all the colors of the visible spectrum, including red, orange, yellow, green, blue, indigo, and violet. When this light enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2), as well as aerosols like dust, water vapor, and pollutants.\n\nThe interaction between the incident sunlight and these atmospheric particles is governed by a fundamental principle in physics known as Rayleigh scattering. This process, named after Lord Rayleigh, who first described it in the late 19th ce

# Start Online Chat Bot Application Using Gradio by using ngrok URL

In [34]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.2 MB/

# The Chatbot Shows the Verbose Output, Without Past Context History

In [42]:
import requests
import json
import gradio as gr
import time

# Function to convert nanoseconds to seconds
def ns_to_sec(ns):
    return round(ns / 1_000_000_000, 4)

# Function to handle API request and return the response and verbose metrics
def query_ollama(prompt):
    # Replace this with the public URL provided by Ngrok
    # public_url = "YOUR_NGROK_URL"  # Replace with your Ngrok URL
    url = f'{public_url}/api/chat'

    # API request payload
    payload = {
        "model": "llama3",  # Replace with the correct model name
        "temperature": 0.6,
        "stream": False,
        "messages": [
            {"role": "system", "content": "You are an AI assistant! Provide accurate, verbose responses."},
            {"role": "user", "content": prompt}
        ]
    }

    # Send the request and time the response
    start_time = time.time()
    response = requests.post(url, json=payload)
    end_time = time.time()
    response_time = round(end_time - start_time, 4)

    response_content = response.content.decode('utf-8')

    # Initialize verbose metrics
    verbose_metrics = {
        "Model Name": "Unknown",
        "Total Duration": "Unknown",
        "Load Duration": "Unknown",
        "Prompt Eval Count": "Unknown",
        "Prompt Eval Duration": "Unknown",
        "Prompt Eval Rate": "Unknown",
        "Eval Count": "Unknown",
        "Eval Duration": "Unknown",
        "Eval Rate": "Unknown",
        "Response Time": f"{response_time}s"
    }

    # Try to parse and handle different response scenarios
    try:
        message_dict = json.loads(response_content)

        if 'message' in message_dict:
            response_message = message_dict['message']['content']
            model_name = message_dict.get('model', 'Unknown')
            total_duration_ns = message_dict.get('total_duration', 0)
            load_duration_ns = message_dict.get('load_duration', 0)
            prompt_eval_count = message_dict.get('prompt_eval_count', 'Unknown')
            prompt_eval_duration_ns = message_dict.get('prompt_eval_duration', 0)
            eval_count = message_dict.get('eval_count', 'Unknown')
            eval_duration_ns = message_dict.get('eval_duration', 0)

            # Convert durations from nanoseconds to seconds
            total_duration = ns_to_sec(total_duration_ns) if total_duration_ns else 'Unknown'
            load_duration = ns_to_sec(load_duration_ns) if load_duration_ns else 'Unknown'
            prompt_eval_duration = ns_to_sec(prompt_eval_duration_ns) if prompt_eval_duration_ns else 'Unknown'
            eval_duration = ns_to_sec(eval_duration_ns) if eval_duration_ns else 'Unknown'

            # Calculate evaluation rates
            prompt_eval_rate = round(prompt_eval_count / prompt_eval_duration, 2) if prompt_eval_count != 'Unknown' and prompt_eval_duration != 'Unknown' else 'Unknown'
            eval_rate = round(eval_count / eval_duration, 2) if eval_count != 'Unknown' and eval_duration != 'Unknown' else 'Unknown'

            # Update verbose metrics
            verbose_metrics = {
                "Model Name": model_name,
                "Total Duration": f"{total_duration}s",
                "Load Duration": f"{load_duration}s",
                "Prompt Eval Count": f"{prompt_eval_count} token(s)",
                "Prompt Eval Duration": f"{prompt_eval_duration}s",
                "Prompt Eval Rate": f"{prompt_eval_rate} tokens/s",
                "Eval Count": f"{eval_count} token(s)",
                "Eval Duration": f"{eval_duration}s",
                "Eval Rate": f"{eval_rate} tokens/s",
                "Response Time": f"{response_time}s"
            }

            return response_message, verbose_metrics

        else:
            return "Response does not contain 'message'. Full response:", response_content

    except json.JSONDecodeError as e:
        return f"Error decoding JSON: {e}", verbose_metrics

    except Exception as e:
        return f"An unexpected error occurred: {e}", verbose_metrics

# Chat Function with Gradio
def chat_interface(history, user_message):
    if user_message.strip() == "":
        return history, "", {}

    response, metrics = query_ollama(user_message)

    # Add user message and bot response to the chat history
    history.append(("User", user_message))
    history.append(("Ollama Bot", response))

    return history, "", metrics

# Gradio interface setup
with gr.Blocks() as demo:
    gr.Markdown("## Ollama Assistant Chat")

    chat_history = gr.State([])
    chatbot = gr.Chatbot(label="Chat History")
    prompt = gr.Textbox(label="Your message:")
    send_button = gr.Button("Send")

    metrics = gr.JSON(label="Verbose Metrics", value={})

    # Chat interaction
    send_button.click(chat_interface, inputs=[chat_history, prompt], outputs=[chatbot, prompt, metrics])

# Launch Gradio app
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6c037bc6a8205a69a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Chat Bot Works With Memory Context Capacity

In [44]:
import requests
import json
import gradio as gr
import time

# Global memory to store conversation history
conversation_history = []

# Function to convert nanoseconds to seconds
def ns_to_sec(ns):
    return round(ns / 1_000_000_000, 4)

# Function to handle API request and return the response and verbose metrics
def query_ollama(prompt):
    global conversation_history
    # Replace this with the public URL provided by Ngrok
    # public_url = "YOUR_NGROK_URL"  # Replace with your Ngrok URL
    url = f'{public_url}/api/chat'

    # Initialize the conversation history for the request
    messages = [
        {"role": "system", "content": "You are an AI assistant! Provide accurate, verbose responses."}
    ] + conversation_history + [{"role": "user", "content": prompt}]

    # API request payload
    payload = {
        "model": "llama3",  # Replace with the correct model name
        "temperature": 0.6,
        "stream": False,
        "messages": messages
    }

    # Send the request and time the response
    start_time = time.time()
    response = requests.post(url, json=payload)
    end_time = time.time()
    response_time = round(end_time - start_time, 4)

    response_content = response.content.decode('utf-8')

    # Initialize verbose metrics
    verbose_metrics = {
        "Model Name": "Unknown",
        "Total Duration": "Unknown",
        "Load Duration": "Unknown",
        "Prompt Eval Count": "Unknown",
        "Prompt Eval Duration": "Unknown",
        "Prompt Eval Rate": "Unknown",
        "Eval Count": "Unknown",
        "Eval Duration": "Unknown",
        "Eval Rate": "Unknown",
        "Response Time": f"{response_time}s"
    }

    # Try to parse and handle different response scenarios
    try:
        message_dict = json.loads(response_content)

        if 'message' in message_dict:
            response_message = message_dict['message']['content']
            model_name = message_dict.get('model', 'Unknown')
            total_duration_ns = message_dict.get('total_duration', 0)
            load_duration_ns = message_dict.get('load_duration', 0)
            prompt_eval_count = message_dict.get('prompt_eval_count', 'Unknown')
            prompt_eval_duration_ns = message_dict.get('prompt_eval_duration', 0)
            eval_count = message_dict.get('eval_count', 'Unknown')
            eval_duration_ns = message_dict.get('eval_duration', 0)

            # Convert durations from nanoseconds to seconds
            total_duration = ns_to_sec(total_duration_ns) if total_duration_ns else 'Unknown'
            load_duration = ns_to_sec(load_duration_ns) if load_duration_ns else 'Unknown'
            prompt_eval_duration = ns_to_sec(prompt_eval_duration_ns) if prompt_eval_duration_ns else 'Unknown'
            eval_duration = ns_to_sec(eval_duration_ns) if eval_duration_ns else 'Unknown'

            # Calculate evaluation rates
            prompt_eval_rate = round(prompt_eval_count / prompt_eval_duration, 2) if prompt_eval_count != 'Unknown' and prompt_eval_duration != 'Unknown' else 'Unknown'
            eval_rate = round(eval_count / eval_duration, 2) if eval_count != 'Unknown' and eval_duration != 'Unknown' else 'Unknown'

            # Update verbose metrics
            verbose_metrics = {
                "Model Name": model_name,
                "Total Duration": f"{total_duration}s",
                "Load Duration": f"{load_duration}s",
                "Prompt Eval Count": f"{prompt_eval_count} token(s)",
                "Prompt Eval Duration": f"{prompt_eval_duration}s",
                "Prompt Eval Rate": f"{prompt_eval_rate} tokens/s",
                "Eval Count": f"{eval_count} token(s)",
                "Eval Duration": f"{eval_duration}s",
                "Eval Rate": f"{eval_rate} tokens/s",
                "Response Time": f"{response_time}s"
            }

            # Update global conversation history
            conversation_history.append({"role": "user", "content": prompt})
            conversation_history.append({"role": "assistant", "content": response_message})

            return response_message, verbose_metrics

        else:
            return "Response does not contain 'message'. Full response:", response_content

    except json.JSONDecodeError as e:
        return f"Error decoding JSON: {e}", verbose_metrics

    except Exception as e:
        return f"An unexpected error occurred: {e}", verbose_metrics

# Chat Function with Gradio
def chat_interface(history, user_message):
    if user_message.strip() == "":
        return history, "", {}

    response, metrics = query_ollama(user_message)

    # Add user message and bot response to the chat history
    history.append(("User", user_message))
    history.append(("Ollama Bot", response))

    return history, "", metrics

# Gradio interface setup
with gr.Blocks() as demo:
    gr.Markdown("## Ollama Assistant Chat")

    chat_history = gr.State([])
    chatbot = gr.Chatbot(label="Chat History")
    prompt = gr.Textbox(label="Your message:")
    send_button = gr.Button("Send")

    metrics = gr.JSON(label="Verbose Metrics", value={})

    # Chat interaction
    send_button.click(chat_interface, inputs=[chat_history, prompt], outputs=[chatbot, prompt, metrics])

# Launch Gradio app
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://12964913288344e440.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
